# Basic-Fit analysis

The Basic-Fit analysis project aims to gain valuable insights from open-source data about all Basic-Fit gyms. The primary goal is to analyze the ratings, number of reviews, comments, and sentiment across all branches. By doing so, this project can help the business better understand end-users' problems and identify any weak points in some branches.

With this information, Basic-Fit can make data-driven decisions to improve customer satisfaction and address any issues that may arise. The project could potentially identify areas where branches are performing exceptionally well and use those as examples for other branches to follow. Overall, the project's findings can provide valuable insights into Basic-Fit's performance and help the business improve its operations to better serve its customers.

## Set up configurations

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import os

In [6]:
#set up configuraitons
options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--headless")  # Required for Codespaces
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

#create a list of Countries and Cities
#countries_cities = {"Spain" : ['Madrid', 'Barcelona', 'Valencia', 'Seville', 'Zaragoza', 'Málaga', 'Murcia', 'Palma', 'Las Palmas de Gran Canaria', 'Bilbao', 'Alicante', 'Córdoba', 'Valladolid', 'Vigo', 'Gijón', 'L Hospitalet de Llobregat', 'A Coruña', 'Vitoria-Gasteiz', 'Granada', 'Elche']}
countries_cities = {"Spain" : ['Madrid', 'Barcelona', 'Valencia', 'Seville', 'Zaragoza', 'Málaga', 'Murcia', 'Palma', 'Las Palmas de Gran Canaria', 'Bilbao', 'Alicante', 'Córdoba', 'Valladolid', 'Vigo']}

In [7]:
#create Dataframe for collecting data
if os.path.isfile("basic-fit.csv"):
    #read file
    df = pd.read_csv(
        "basic-fit.csv",
        dtype={
            "Name" : str,
            "Country" : str,
            "City" : str,
            "Address" : str,
            "Rating" : float,
            "Reviews" : int,
            "Site" : str,
            "Link" : str
        }
        )
else:
    # create an empty dataframe with columns
    df = pd.DataFrame({
        "Name" : pd.Series(dtype="str"),
        "Country" : pd.Series(dtype="str"),
        "City" : pd.Series(dtype="str"),  
        "Address" : pd.Series(dtype="str"),
        "Rating" : pd.Series(dtype="float"),
        "Reviews" : pd.Series(dtype="int"),
        "Site" : pd.Series(dtype="str"), 
        "Link" : pd.Series(dtype="str")
        })

## Defining the question

1. What is the overall customer satisfaction rating for Basic-Fit chains?
2. How does the customer satisfaction rating vary across different Basic-Fit chains?
3. What are the most common complaints and positive feedback from customers about Basic-Fit chains?
4. How does the customer satisfaction rating correlate with the location, size or age of the Basic-Fit chain?
5. Are there any notable differences in customer satisfaction between Basic-Fit chains in different countries or regions?
6. How does Basic-Fit chains compare with other fitness chains in terms of customer satisfaction rating?
7. What are the factors that drive customer satisfaction for Basic-Fit chains?
8. How has the customer satisfaction rating for Basic-Fit chains changed over time?
9. What improvements could Basic-Fit chains make to enhance customer satisfaction?
10. Are there any patterns or trends in the customer feedback for Basic-Fit chains that can be used to inform future business decisions?

## Collecting the data

In [8]:
#scrape data from the google map
for country, cities in countries_cities.items():
    for city in cities:
        driver = webdriver.Chrome(options=options)
        #driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.get(f"https://www.google.com/maps/search/basic-fit+near+{city}+in+{country}/")

        #accept offerta
        accept_button = driver.find_elements("xpath", "//button[@aria-label='Accept all']")[0]
        accept_button.click()

        #find branches ellements
        branches = driver.find_elements("xpath", "//div[contains(@aria-label,'Basic-Fit') and @class='Nv2PK tH5CWc THOPZb ']")
        
        #get data from the element
        for branch in branches:

            #branch name
            branch_name = branch.get_attribute("aria-label")

            #branch detaled information link
            detailed_web = branch.find_element("xpath", ".//a[@class='hfpxzc']")
            detailed_web_link = detailed_web.get_attribute("href")

            #branch adress
            branch_address_el = branch.find_element("xpath", ".//div[@class='W4Efsd']/span[2]/jsl/span[2]")
            branch_address = branch_address_el.get_attribute("innerHTML")

            #branch rating
            rating = branch.find_element("xpath", ".//div[@class='AJB7ye']/span[2]/span[2]")
            rating_text = rating.get_attribute("aria-label")
            stars = rating_text.split("stars")[0].strip()

            #branch reviews
            reviews = rating_text.split("stars")[1].strip().split(" ")[0].replace(",", "")

            #branch web site
            branch_web = branch.find_element("xpath", ".//a[@class='lcr4fd S9kvJb']")
            branch_web_link = branch_web.get_attribute("href")

            #create a new row
            new_row = {
                    "Name" : f"{branch_name}",
                    "Country" : f"{country}",
                    "City" : f"{city}",
                    "Address" : f"{branch_address}",
                    "Rating" : f"{stars}",
                    "Reviews" : f"{reviews}",
                    "Site" : f"{branch_web_link}",
                    "Link" : f"{detailed_web_link}"
                }

            #add the row to the datafame
            df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

# Close the browser window
driver.quit()

## Cleaning the data

In [9]:
#get dataframe shape
df.shape

(41, 8)

In [10]:
#check column datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Name     41 non-null     object 
 1   Country  41 non-null     object 
 2   City     41 non-null     object 
 3   Address  41 non-null     object 
 4   Rating   41 non-null     float64
 5   Reviews  41 non-null     int64  
 6   Site     41 non-null     object 
 7   Link     41 non-null     object 
dtypes: float64(1), int64(1), object(6)
memory usage: 2.7+ KB


In [11]:
#change dtypes
df["Rating"] = df["Rating"].astype(float)
df["Reviews"] = df["Reviews"].astype(int)
df.dtypes

Name        object
Country     object
City        object
Address     object
Rating     float64
Reviews      int64
Site        object
Link        object
dtype: object

In [12]:
#get data sample
df.head()

,Name,Country,City,Address,Rating,Reviews,Site,Link
0,Basic-Fit Madrid Goya,Spain,Madrid,"Calle de Goya, 43",3.9,524,https://www.google.com/aclk?sa=l&ai=DChcSEwi-1...,https://www.google.com/maps/place/Basic-Fit+Ma...
1,Basic-Fit,Spain,Madrid,"C. de Atocha, 24",3.5,1000,https://www.basic-fit.com/es-es/gimnasio/basic...,https://www.google.com/maps/place/Basic-Fit/da...
2,Basic-Fit Madrid Núñez de Balboa,Spain,Madrid,"Calle de Núñez de Balboa, 115",4.0,600,https://www.basic-fit.com/es-es/gimnasio/basic...,https://www.google.com/maps/place/Basic-Fit+Ma...
3,Basic-Fit,Spain,Madrid,"C. de la Montera, 41",3.6,535,https://www.basic-fit.com/es-es/gimnasio/basic...,https://www.google.com/maps/place/Basic-Fit/da...
4,Basic-Fit Madrid Sor Ángela de la Cruz,Spain,Madrid,"C. de Sor Ángela de la Cruz, 24",3.4,773,https://www.basic-fit.com/es-es/gimnasio/basic...,https://www.google.com/maps/place/Basic-Fit+Ma...


In [13]:
#describe data
df.describe()

,Rating,Reviews
count,41.000000,41.000000
mean,4.329268,330.634146
std,0.484894,304.876102
min,3.300000,6.000000
25%,3.900000,69.000000
50%,4.300000,236.000000
75%,4.800000,524.000000
max,4.900000,1135.000000


In [14]:
#amount of unique categories
df["Address"].nunique()

41

In [15]:
#find duplicates
duplicates = (df.groupby(by=["Address"])
    .count()["Name"]
    .sort_values(ascending=False)
    )
    
duplicates.loc[duplicates > 1]

Series([], Name: Name, dtype: int64)

In [16]:
#drop duplicates
df_unique = df.drop_duplicates(
    subset=["Name", "Address"],
    keep="first",
    inplace=False
    ).reset_index(drop=True)

#get dataframe shape
df_unique.shape

(41, 8)

In [17]:
#save data to csv
df_unique.to_csv("basic-fit.csv", index=False)